# Day 17: Pyroclastic Flow 

It's basically tetris

Rocks: 5 shapes
```
rock 0
####

rock 1
.#.
###
.#.

rock 2
..#
..#
###

rock 3
#
#
#
#

rock 4
##
##
```

The tall, vertical chamber is exactly seven units wide. 

Each rock appears so that its left edge is two units away from the left wall and its bottom edge is three units above the highest rock in the room (or the floor, if there isn't one).

After a rock appears, it alternates between being pushed by a jet of hot gas one unit (in the direction indicated by the next symbol in the jet pattern) and then falling one unit down.


Result we're looking for is *height of rock tower after 2022 rocks have falled*


In [ ]:
from pprint import pprint
import copy
from math import floor
#test data
testJetPattern = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

class Rock:
    def __init__(self, ascii:str):
        self.ascii = ascii
        self.height = 0
        self.rows = []
        for row in ascii.splitlines():
            self.height += 1
            self.width = len(row)
            line = []
            for c in list(row):
                if c == '#':
                    line.append(1)
                else:
                    line.append(0)
            self.rows.insert(0,line) #input in pictorial view.. so we need to flip the rows... i.e. last row becomes bottom row of our shape

    @classmethod
    def newShape(cls,shape:int):
        match shape:
            case 0:
                rock = '####'
            case 1:
                rock = """.#.
###
.#."""
                
            case 2:
                rock = """..#
..#
###"""

            case 3:
                rock = """#
#
#
#"""

            case 4:
                rock = """##
##"""
            case _:
                raise Exception('invalid shape number')
        return cls(rock)

class Game:
    def __init__(self, cycleDetection:bool=False):
        self.highestRock = 0 #start with the floor
        self.gridWidth = 7
        self.shapeStartLeftPad = 2
        self.shapeStartVerticalPad = 3
        self.currentShape = -1 
        self.shapeCount = 0
        self.rows:[[int]] = [[0] * self.gridWidth] #2D grid, row first (starting at floor) then column... 0=empty, 1=rock
        self.rowsToTrack = 40 #top number of rows we track in the row
        self.bottomRowNumber = 0
        self.shapeRows = [] #2D grid to hold the sprite of the current shape... we can just chop off rows at the head to have it fall
        self.cycleDetection = cycleDetection

    def truncateRows(self):
        #keep the field of play small by removing bottom rows that are no longer active (we're assuming which ones aren't active)
        rowCount = len(self.rows)
        rowsToRemove = rowCount - self.rowsToTrack
        if rowsToRemove > 0:
            self.bottomRowNumber += rowsToRemove
            self.rows = self.rows[-self.rowsToTrack:]

    def nextShape(self):
        #make the next shape fall
        self.currentShape = (self.currentShape + 1) % 5
        self.shapeCount += 1
        rock = Rock.newShape(self.currentShape)

        #truncate the grid
        self.truncateRows()

        #need to add three rows ABOVE THE CURRENT HIGHEST ROCK + height of shape... need the shapes as objects
        rockHeightInRows = self.highestRock - self.bottomRowNumber
        rowsToAdd = rockHeightInRows + 3 - len(self.rows) + rock.height

        for _ in range(rowsToAdd):
            self.rows.append([0] * self.gridWidth)

        #need to build shapeRows and place the shape in the right place
        self.shapeRows = []
        for _ in range(len(self.rows)):
            self.shapeRows.append([0] * self.gridWidth)
        for r in range(rock.height):
            for c in range(rock.width):
                rockValue = rock.rows[r][c]
                if rockValue == 1:
                    rr = r + rockHeightInRows + 3
                    cc = c + self.shapeStartLeftPad
                    self.shapeRows[rr][cc] = 1

    def shiftDown(self)->bool:
        #shift the shape down1 and return true... or, if this would cause a collision then return false
        emptyRow = [0] * self.gridWidth
        #check for being at bottom
        if self.shapeRows[0] != emptyRow:
            return False
        trialShapeRows = self.shapeRows[1:].copy()
        if not self.collision(trialShapeRows):
            self.shapeRows = trialShapeRows
            return True
        else:
            return False
    
    def collision(self,trialShapeRows)->bool:
        #return true if the trialShapeRow would result in collision
        emptyRow = [0] * self.gridWidth
        collision = False
        for r in range(len(trialShapeRows)):
            if trialShapeRows[r]!=emptyRow and not collision: #skip over the empty rows
                for c in range(self.gridWidth):
                    shapeValue = trialShapeRows[r][c]
                    gridValue = self.rows[r][c]
                    if shapeValue==1 and gridValue==1:
                        collision = True
                        break
        return collision

    #OMG... the left or right shift could cause a collision... so need to check for this condition as well... and if it would collide, we don't move.
    def shiftLeft(self)->bool:
        #need to know where the most left part is
        leftmost = self.gridWidth
        for r in range(len(self.shapeRows)):
            for c in range(self.gridWidth):
                shapeValue = self.shapeRows[r][c]
                if shapeValue == 1:
                    leftmost = min(leftmost, c)
                    break
        if leftmost != 0:
            #can shift left...but need to check for collisions
            trailShapeRows = copy.deepcopy(self.shapeRows)

            for r in range(len(self.shapeRows)):
                for c in range(self.gridWidth):
                    if c + 1 < self.gridWidth:
                        trailShapeRows[r][c] = trailShapeRows[r][c+1]
                    else:
                        trailShapeRows[r][c] = 0
            if not self.collision(trailShapeRows):
                self.shapeRows = trailShapeRows
                return True
            else:
                return False
        else:
            return False

    def shiftRight(self):
        #need to know where right most part is
        rightmost = 0
        for r in range(len(self.shapeRows)):
            for c in range(self.gridWidth):
                shapeValue = self.shapeRows[r][c]
                if shapeValue == 1:
                    rightmost = max(rightmost, c)
        if rightmost != self.gridWidth-1:
            #can shift right
            trailShapeRows = copy.deepcopy(self.shapeRows)
            for r in range(len(trailShapeRows)):
                for c in range(self.gridWidth):
                    if c==0:
                        trailShapeRows[r][c] = 0
                    else:
                        trailShapeRows[r][c] = self.shapeRows[r][c-1]
            if not self.collision(trailShapeRows):
                self.shapeRows = trailShapeRows
                return True
            else:
                return False
        else:
            return False

    def solidifyShape(self):
        emptyRow = [0] * self.gridWidth
        for r in range(len(self.shapeRows)):
            if self.shapeRows[r]!=emptyRow : #skip over the empty rows
                for c in range(self.gridWidth):
                    shapeValue = self.shapeRows[r][c]
                    if shapeValue==1:
                        self.rows[r][c] = 1
        #need to update highest rock
        self.highestRock = 0
        for r in range(len(self.rows)):
            if self.rows[r] != emptyRow:
                self.highestRock = r+1 + self.bottomRowNumber
        
    def cycleDetectionKey(self, jetpatternCount:int):
        #key is a composite of current shape, jet pattern and board state (needs encoding)
        #how to encode current grid state? It's a massive array of bits - turn it into a number!
        gridState = 0
        for r in self.rows:
            for bit in r:
                gridState = (gridState << 1) | bit
        return gridState * 1000000 + self.currentShape*100000 + jetpatternCount

    def run(self, jetpatternStr:str, numRocks:int)->int:
        #return the height of the tower after x rocks
        jetpattern = list(jetpatternStr)
        jetpatternCount = 0
        

        #record these, so we can spot when we find again
        jpshapekey = self.currentShape*100000 + jetpatternCount
        cyclicSet = {}
        cyclicSet[jpshapekey] = (0,0) #(shape number, rock height)
        cycleDetected = False

        self.nextShape()
        #alternates between being pushed by a jet of hot gas one unit (in the direction indicated by the next symbol in the jet pattern) and then falling one unit down
        while self.shapeCount <= numRocks:
            #if end of jet pattern reached, it repeats
            if len(jetpattern)==0:
                print('### jet pattern depleted ###')
                jetpattern = list(jetpatternStr)
                jetpatternCount = 0
            
            shiftDir = jetpattern.pop(0)
            jetpatternCount += 1
            if shiftDir == '>':
                self.shiftRight()
            elif shiftDir == '<':
                self.shiftLeft()
            else:
                raise Exception('Unexpected jet:'+shiftDir)
            #print('Jet ' + shiftDir +' --------')
            #self.printRows()
            if not self.shiftDown():
                #shape can't be dropped and needs to be solidified
                self.solidifyShape()

                #print('Last blast direction = ' + shiftDir)
                print('After ' + str(self.shapeCount) + ' rocks, tower height is '+str(self.highestRock) + ' remaining jet pattern is '+ str(len(jetpattern)))
             
                #detect if we've been here before
                jpshapekey = self.cycleDetectionKey(jetpatternCount)
                if jpshapekey in cyclicSet and self.cycleDetection:
                    print('#### Cycle detected ####')
                    cycleDetected = True
                    #how do we get from here to result... 
                    #well... we could skip x number of shapes 
                    #but we also need to know how many more steps are needed before this shape clicks into place...
                    preambleShapes = cyclicSet[jpshapekey][0]
                    preambleHeight = cyclicSet[jpshapekey][1]
                    shapesPerCycle = self.shapeCount - preambleShapes
                    heightPerCycle = self.highestRock - preambleHeight
                    remainingRocks = numRocks - self.shapeCount
                    remainingCycle = floor(remainingRocks / shapesPerCycle)
                    print('Skipping '+str(remainingCycle)+' cycles')
                    #self.highestRock += heightPerCycle * remainingCycle #don't do this here... it kills the performance of the last few shapes
                    self.shapeCount += shapesPerCycle * remainingCycle
                    print('After ' + str(self.shapeCount) + ' rocks, tower height is '+str(self.highestRock) + ' remaining jet pattern is '+ str(len(jetpattern)))
                    cyclicSet.clear()
                else:
                    cyclicSet[jpshapekey] = (self.shapeCount,self.highestRock)
                
                #self.printRows()
                self.nextShape()
                #the above line is causing a performance issue, after we've faked the amounts... we clearly need to save the row height and add in at the end.
        #end of while, so we're done
        if self.cycleDetection:
            self.highestRock += heightPerCycle * remainingCycle

            #print('Drop --------')
            #self.printRows()
            
        
        return self.highestRock

    def printRows(self):
        outputRows = []
        for r in range(len(self.rows)):
            output = "|"
            for c in range(self.gridWidth):
                outputVal = 0
                gridVal = self.rows[r][c]
                if r < len(self.shapeRows):
                    shapeVal = self.shapeRows[r][c]
                    if shapeVal == 1:
                        outputVal = 2
                if gridVal == 1:
                    outputVal = 1
                match outputVal:
                    case 2:
                        output = output + '@'
                    case 1:
                        output = output + '#'
                    case 0:
                        output = output + '.'
            output = output + '|'
            outputRows.append(output)
        outputRows.reverse()
        for r in outputRows:
            print(r)
        




##unit tests
# r0 = Rock.newShape(0)
# print(r0.rows)
# r2 = Rock.newShape(2)
# print(r2.rows)
# print(r2.rows[0][2])


# g = Game()
# for i in range(4):
#     print('----')
#     g.nextShape()
#     pprint(g.shapeRows)
#     print(g.shiftDown())
#     print(g.shiftRight())
#     print(g.shiftRight())
#     print(g.shiftRight())
#     print(g.shiftRight())
#     print(g.shiftRight())
#     pprint(g.shapeRows)






In [ ]:
#part 1 tests
g = Game()
testResult = g.run(testJetPattern,2022)
print(testResult)
print(testResult==3068)


In [ ]:
#tests
g = Game(cycleDetection=True)
testResult = g.run(testJetPattern,1000000000000)
print(testResult)
print(testResult==1514285714288)


In [ ]:
#using puzzle input
input = open('day17input.txt').read()

#Ah, the input has a new line character. But it's all on one line... easyfix.
input = input.splitlines()[0]
gg = Game(cycleDetection=True)
print(gg.run(input,1000000000000)) 

#part 2 - let's see if we can get good enough answer to extrapolate
#print(gg.scoreboard)



In [ ]:
# part 2 rough calcs
timePerRock = 2022/30
totalRocks = 1000000000000
totalTime = totalRocks * timePerRock
print(totalTime/(60*60*24*365))

#how many loops of the input would this be?
input = open('day17input.txt').read()
input = input.splitlines()[0]
print(len(input))
print(totalRocks/len(input))


# Part 2 - thoughts

The above maths shows this would take 2137239 YEARS to run a simulation this long.
We would also loop through the input 99098206 times.

Is there some sort of cycle? That's the thing to find... 

OK.. done that. Worked for the test. 
For puzzle input I got
1558857142871

Apparently this is too low.
What have I missed?
Board state! Need to track this. Can probably get away with the top N rows of the grid.

